# MCDA 코드 분석

### 에이전트가 공간 행동에 MCDA(Multi Criteria Decision Analyses)를 활용할 수 있도록 합니다.

### Input:

lattice; 태양 / 하늘 / 거리 접근 / 입구 접근  특정 부분의 CSV 파일
excel program mtx, pref 정보 pref 테이블과 mtx 포함된 excel data.

### Output:

Agent based modeling lattice MCDA (성장 저장)

### 1. 기본 env  <br>

1. 기본 env load
2. lattice_from_csv = csv에서 lattice 배열 만드는 함수 정의 <br>
3. tri_to_pv = 삼각형 메시(tri_mesh)를 PyVista 형식으로 변환 <br>
4. stencil 이웃 정의

In [2]:
#env 로드
import os
import topogenesis as tg
import trimesh as tm
import numpy as np
import pyvista as pv
import panel as pn
import pandas as pd 
import copy

import ipywidgets as widgets
from ladybug.sunpath import Sunpath #ladybug sunpath 호출
from IPython.display import display, Image
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import MinMaxScaler

# csv에서 lattice 배열 만드는 함수 정의
def lattice_from_csv(file_path):
    # 데이터를 읽어옴
    meta_df = pd.read_csv(file_path, nrows=3)
    # shape, unit, minbound 데이터를 배열로 변환
    shape = np.array(meta_df['shape'])       # 시작 좌표 [x_min, y_min, z_min] 예를 들어, [-55000, -7000, 0] x축 -55000, y축 -7000, z축 0의 위치
    unit = np.array(meta_df['unit'])         # 몇개의 셀로 구성되는지 [x_shape, y_shape, z_shape] 예를 들어 [56, 35, 24]이면 x축 56개, y 35개, z 24개의 셀을 가집니다.
    minbound = np.array(meta_df['minbound']) # 각 셀의 크기. 예를 들어, [1000, 1000, 3000] 각 셀의 크기는 x축 1000 단위, y축 1000 단위, z축 3000 단위
    # np.array는 csv에서 불러온 데이터를 배열로 변환하는데 사용 
    # 그리드 데이터를 읽어옴
    lattice_df = pd.read_csv(file_path, skiprows=5) # 앞의 5개 줄을 스킵하고 6개 부터 데이터를 불러오기
    # 데이터의 'value' 열을 배열로 변환한 후 이를 'shape' 배열에 맞게 재구성 
    buffer = np.array(lattice_df['value']).reshape(shape)
    # lattice를 토포제네시스에서 재구성 buffer는 배열에 맞는 value값. 
    l = tg.to_lattice(buffer, minbound=minbound, unit=unit)
    
    return l

# tri_to_pv 함수 정의: 삼각형 메시(tri_mesh)를 PyVista 형식으로 변환합니다.
def tri_to_pv(tri_mesh):
    #  tri_mesh.faces 배열에 각 face의 시작 부분에 3을 추가하여 새로운 배열을 만듭니다.
    # 'constant' 모드로 패딩을 추가하여 각 face의 시작에 3을 삽입합니다.
    faces = np.pad(tri_mesh.faces, ((0, 0), (1, 0)), 'constant', constant_values=3)
    # PyVista의 PolyData 객체를 생성하여, tri_mesh의 정점(vertices)과 패딩된 faces 배열을 사용합니다.
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)    
    # 변환된 PyVista 메쉬 객체를 반환합니다.
    return pv_mesh

# 1이라는 범위에 대한 이웃패턴 정의 (중심을 포함함)
stencil = tg.create_stencil("von_neumann", 1, 1)
# 중심을 인덱스 비활성화 - [0,0,0]을 0으로 세팅함으로 중심을 패턴 비활성화
stencil.set_index([0,0,0], 0)


### 2. CSV OBJ Load<br>

1. topo에서 만든 lattice 격자파일 load <br>
2. 주변환경 obj <br>
3. street point csv <br>
4. entrance point csv <br>


In [3]:
#csv 로드
csv_path = os.path.relpath('original_lattice/interior_lattice.csv')
#obj 포인트 csv 로드
context_mesh = tm.load(os.path.relpath('obj\context.obj'))  # 주변환경 obj 메시 불러오기
street_pc = tg.cloud_from_csv("pts\pts_str(11).CSV")        # street point as csv (포인트를 csv로 저장) 
entrance_pc = tg.cloud_from_csv("pts\ent_pts01.CSV")        # entrance point as csv (라이노 포인트를 csv로 저장)
envelope_lattice = tg.lattice_from_csv(csv_path)

avail_lattice = tg.lattice_from_csv(csv_path)                                         # CSV 파일에서 격자 데이터 불러오기
init_avail_lattice = tg.to_lattice(np.copy(avail_lattice), avail_lattice)             # 복사된 데이터를 격자 객체로 변환

### 3. Data 추출 pd

1. 선택한 열을 복사하여 program_prefs 데이터프레임 생성 <br><br>
   [sun_acc] [태양 근접 선호도]<br>
   [ent_acc] [입구 근접 선호도]<br>
   [str_acc] [도로 근접 선호도]<br>
   [ung_pre] [지하 선호도] <br>
   [top_pre] [탑층 선호도] <br><br>

2. program_pref 필드 생성 
   제작한 lattice 필드에서  field에 딕셔너리 저장 

3. program_mtx 필드 생성
   

In [4]:
# 'excel/program (이노션).xlsx' 파일에서 'Sheet1' 시트를 읽어옴
df = pd.read_excel('excel/program (이노션).xlsx', sheet_name='Sheet1')
program_prefs = df[['sun_acc', 'ent_acc', 'str_acc', 'ung_pre', 'dist_facade','top_pre']].copy()

# 인덱스를 재설정(기존 index를 버리고 새로운 인덱스로) 0부터 숫자 인덱스를 생성
program_prefs.reset_index(drop=True, inplace=True)
fields = {}  # 필드를 저장할 빈 딕셔너리를 생성

for f in program_prefs.columns:  # 각 필드에 대한 파일 경로 설정    
    lattice_path = os.path.relpath('matrixs/' + f + '.csv')      
    try:        
        fields[f] = tg.lattice_from_csv(lattice_path)  # 배열 파일에서 데이터를 불러와서 fields 딕셔너리에 저장
    except:        
        fields[f] = copy.deepcopy(avail_lattice * 0 + 1)# 파일이 없을 경우, 모든 값을 1로 설정한 기본 격자 데이터를 fields 딕셔너리에 저장


top_pre_index = df.columns.get_loc('top_pre') # 'top_pre' 열의 인덱스 찾기 (5번째에 위치)
program_mtx = df.iloc[:, top_pre_index + 1:]  # 'top_pre' 열 이후 모든 열을 포함하는 새로운 데이터프레임 생성 (프로그램 상관관계 matrix 찾기)
program_mtx.columns = range(len(program_mtx.columns)) # 새로운 데이터프레임의 열 이름을 0부터 시작하는 숫자로 변경     (프로그램 번호입력 0~ ) 

space_list = df['space_name'].to_dict() # 'space_name' 열을 딕셔너리로 변환하여 'space_list'에 저장
sizes_complete = df['vox_amount']# 'vox_amount' 열의 값을 가져와 'sizes_complete'에 저장

agent_areas = [] # 'agent_areas' 리스트 초기화

for area in sizes_complete: # 'sizes_complete'의 각 값에 대해 반복 
    agent_areas.append(round(int(area)))    # 각 값을 정수로 변환하고 반올림하여 'agent_areas'에 추가

print(space_list)   # 변환된 딕셔너리를 출력
print(agent_areas)  # 'agent_areas' 리스트를 출력
print(program_mtx)

{0: 'Lobby', 1: 'Branding Space', 2: 'External Contact', 3: 'Interview Room', 4: 'Personal Workspace', 5: 'Work Support Area', 6: 'Personal Locker Room', 7: 'Executive Space', 8: 'VIP Reception', 9: 'Secretary Room', 10: 'Pantry', 11: 'Executive Meeting', 12: 'Executive Offices', 13: 'Focus Rooms', 14: 'Project Rooms', 15: 'Conference Rooms', 16: '4-Person Meeting', 17: '6-Person Meeting', 18: '8-Person Meeting', 19: '12-Person Meeting', 20: '18-Person Meeting', 21: 'Idea Room', 22: 'Mail Room', 23: 'Oasis', 24: 'PT Practice Room', 25: 'Screening Room', 26: 'Studio', 27: 'Video Editing', 28: 'Plotter/Board Room', 29: 'Training Room', 30: 'Ino Kitchen', 31: 'Kitchen', 32: 'Ino Cafe', 33: 'Fitness Facilities', 34: 'Dressing/Shower', 35: 'Counseling Room', 36: 'Personal Rest Room', 37: 'Sleeping Room', 38: 'Nursing Room', 39: 'Work Lounge', 40: 'Town Hall', 41: 'Seminar Room', 42: 'Library', 43: 'Social Space', 44: 'Outdoor Terrace', 45: "Driver's Waiting", 46: 'Server Room', 47: 'Cleanin

이 코드는 여러 에이전트(사람 또는 객체)가 있는 격자(그리드) 안에 최적의 위치를 찾는 작업을 합니다.  
각 에이전트는 자신의 선호도에 따라 특정 위치를 더 좋아하고, 다른 에이전트와의 상관관계를 반영하여 위치를 선택합니다.  
각 층에는 최대 배치할 수 있는 에이전트 수가 제한되어 있습니다.


In [7]:
import numpy as np

# 고정된 난수 시드를 설정하여 항상 동일한 결과를 얻을 수 있도록 합니다.
np.random.seed(42)

# 각 층에 배치할 수 있는 최대 에이전트 수
max_agents_per_floor = 6
# 가용 격자를 모두 -1로 초기화 (모든 셀을 사용 불가능으로 설정)
occ_lattice = avail_lattice * 0 - 1
# 초기 점유 격자의 복사본 생성
init_occ_lattice = np.copy(occ_lattice)

# 각 에이전트의 위치를 저장할 리스트 초기화
agn_locs = [None] * len(program_prefs)
# 각 층의 에이전트 수를 추적하는 배열 초기화
floor_counts = np.zeros(avail_lattice.shape[2], dtype=int)

def calculate_weighted_average(values, weights):
    return sum(v * w for v, w in zip(values, weights)) / sum(weights)

for a_id, a_prefs in program_prefs.iterrows():
    pref_lattice = avail_lattice * 0.0
    for f, w in a_prefs.items():
        pref_lattice += fields[f] * w

    flat_pref_lattice = pref_lattice.flatten()
    
    # 모든 가용한 위치의 인덱스를 가져옴
    all_indices = np.where(flat_pref_lattice > 0)[0]
    
    # 각 가용한 위치에 대해 프로그램 간 상관관계를 반영하여 선호도 조정
    all_scores = flat_pref_lattice[all_indices].copy()
    adjustment_factors = np.ones_like(all_scores)
    num_adjustments = np.zeros_like(all_scores)  # 각 위치에 대한 상관관계 적용 횟수를 추적

    for other_id in range(len(program_prefs)):
        if other_id != a_id and agn_locs[other_id] is not None:
            corr = program_mtx.iloc[a_id, other_id]  # DataFrame에서 상관관계 값을 가져옴
            other_loc = agn_locs[other_id]
            for idx, loc in enumerate(np.unravel_index(all_indices, pref_lattice.shape)):
                if tuple(loc) == other_loc:
                    adjustment_factors[idx] += corr
                    num_adjustments[idx] += 1

    # 가중평균 계산: 기본 선호도와 상관관계의 조정값을 가중평균으로 반영
    for i in range(len(all_scores)):
        if num_adjustments[i] > 0:
            all_scores[i] = calculate_weighted_average([all_scores[i], adjustment_factors[i]], [1, num_adjustments[i]])

    selected = False
    a_origin_2 = None
    for select_id in all_indices[np.argsort(-all_scores)]:  # 모든 위치에서 최적의 위치를 선택
        a_origin_1 = np.unravel_index(select_id, avail_lattice.shape)
        # 가용한 위치와 층의 에이전트 수를 고려하여 위치 선정
        if avail_lattice[a_origin_1] != 0 and floor_counts[a_origin_1[2]] < max_agents_per_floor:
            a_origin_2 = a_origin_1
            selected = True
            break

    if not selected:
        # 만약 모든 위치 중에서 적절한 위치를 찾지 못했을 때, 다음으로 높은 값을 찾음
        while not selected:
            select_id = np.argmax(flat_pref_lattice)
            a_origin_1 = np.unravel_index(select_id, avail_lattice.shape)
            if avail_lattice[a_origin_1] != 0 and floor_counts[a_origin_1[2]] < max_agents_per_floor:
                a_origin_2 = a_origin_1
                selected = True
            else:
                flat_pref_lattice[a_origin_1] = -1  # 현재 위치를 -1로 설정하여 다음 최고 값을 선택하도록

    # 에이전트 위치 리스트에 추가
    agn_locs[a_id] = a_origin_2
    # 가용 배열에서 해당 위치를 0으로 설정 (이제 사용 불가능)
    avail_lattice[a_origin_2] = 0
    # 해당 층의 에이전트 수를 증가
    floor_counts[a_origin_2[2]] += 1
    occ_lattice[a_origin_2] = a_id

    print(f"All indices for agent {a_id}: {all_indices}")
    print(f"All values for agent {a_id}: {flat_pref_lattice[all_indices]}")
    print(f"Adjusted all scores for agent {a_id}: {all_scores}")
    print(f"Selected location for agent {a_id}: {a_origin_2}")

# 출력하여 확인
print("Agent locations (agn_locs):")
print(agn_locs)


All indices for agent 0: [    0     1     2 ... 47037 47038 47039]
All values for agent 0: [0.9 0.9 0.9 ... 0.9 0.9 0.9]
Adjusted all scores for agent 0: [0.9 0.9 0.9 ... 0.9 0.9 0.9]
Selected location for agent 0: (31, 8, 5)
All indices for agent 1: [    0     1     2 ... 47037 47038 47039]
All values for agent 1: [0.9 0.9 0.9 ... 0.9 0.9 0.9]
Adjusted all scores for agent 1: [0.9 0.9 0.9 ... 0.9 0.9 0.9]
Selected location for agent 1: (46, 9, 5)
All indices for agent 2: [    0     1     2 ... 47037 47038 47039]
All values for agent 2: [0.9 0.9 0.9 ... 0.9 0.9 0.9]
Adjusted all scores for agent 2: [0.9 0.9 0.9 ... 0.9 0.9 0.9]
Selected location for agent 2: (38, 10, 5)
All indices for agent 3: [    0     1     2 ... 47037 47038 47039]
All values for agent 3: [0.8 0.8 0.8 ... 0.8 0.8 0.8]
Adjusted all scores for agent 3: [0.8 0.8 0.8 ... 0.8 0.8 0.8]
Selected location for agent 3: (40, 10, 5)
All indices for agent 4: [    0     1     2 ... 47037 47038 47039]
All values for agent 4: [0.

In [7]:
import numpy as np

# 각 층에 배치할 수 있는 최대 에이전트 수
max_agents_per_floor = 6
# 가용 격자를 모두 -1로 초기화 (모든 셀을 사용 불가능으로 설정)
occ_lattice = avail_lattice * 0 - 1
# 초기 점유 격자의 복사본 생성
init_occ_lattice = np.copy(occ_lattice)

# 각 에이전트의 위치를 저장할 리스트 초기화
agn_locs = [None] * len(program_prefs)
# 각 층의 에이전트 수를 추적하는 배열 초기화
floor_counts = np.zeros(avail_lattice.shape[2], dtype=int)

def calculate_weighted_average(values, weights):
    return sum(v * w for v, w in zip(values, weights)) / sum(weights)

for a_id, a_prefs in program_prefs.iterrows():
    pref_lattice = avail_lattice * 0.0
    for f, w in a_prefs.items():
        pref_lattice += fields[f] * w

    flat_pref_lattice = pref_lattice.flatten()
    
    # 모든 가용한 위치의 인덱스를 가져옴
    all_indices = np.where(flat_pref_lattice > 0)[0]
    
    # 각 가용한 위치에 대해 프로그램 간 상관관계를 반영하여 선호도 조정
    all_scores = flat_pref_lattice[all_indices].copy()
    adjustment_factors = np.ones_like(all_scores)
    num_adjustments = np.zeros_like(all_scores)  # 각 위치에 대한 상관관계 적용 횟수를 추적

    for other_id in range(len(program_prefs)):
        if other_id != a_id and agn_locs[other_id] is not None:
            corr = program_mtx.iloc[a_id, other_id]  # DataFrame에서 상관관계 값을 가져옴
            other_loc = agn_locs[other_id]
            for idx, loc in enumerate(np.unravel_index(all_indices, pref_lattice.shape)):
                if tuple(loc) == other_loc:
                    adjustment_factors[idx] += corr
                    num_adjustments[idx] += 1

    # 가중평균 계산: 기본 선호도와 상관관계의 조정값을 가중평균으로 반영
    for i in range(len(all_scores)):
        if num_adjustments[i] > 0:
            all_scores[i] = calculate_weighted_average([all_scores[i], adjustment_factors[i]], [1, num_adjustments[i]])

    selected = False
    a_origin_2 = None
    for select_id in all_indices[np.argsort(-all_scores)]:  # 모든 위치에서 최적의 위치를 선택
        a_origin_1 = np.unravel_index(select_id, avail_lattice.shape)
        # 가용한 위치와 층의 에이전트 수를 고려하여 위치 선정
        if avail_lattice[a_origin_1] != 0 and floor_counts[a_origin_1[2]] < max_agents_per_floor:
            a_origin_2 = a_origin_1
            selected = True
            break

    if not selected:
        # 만약 모든 위치 중에서 적절한 위치를 찾지 못했을 때, 다음으로 높은 값을 찾음
        while not selected:
            select_id = np.argmax(flat_pref_lattice)
            a_origin_1 = np.unravel_index(select_id, avail_lattice.shape)
            if avail_lattice[a_origin_1] != 0 and floor_counts[a_origin_1[2]] < max_agents_per_floor:
                a_origin_2 = a_origin_1
                selected = True
            else:
                flat_pref_lattice[a_origin_1] = -1  # 현재 위치를 -1로 설정하여 다음 최고 값을 선택하도록

    # 에이전트 위치 리스트에 추가
    agn_locs[a_id] = a_origin_2
    # 가용 배열에서 해당 위치를 0으로 설정 (이제 사용 불가능)
    avail_lattice[a_origin_2] = 0
    # 해당 층의 에이전트 수를 증가
    floor_counts[a_origin_2[2]] += 1
    occ_lattice[a_origin_2] = a_id

    print(f"All indices for agent {a_id}: {all_indices}")
    print(f"All values for agent {a_id}: {flat_pref_lattice[all_indices]}")
    print(f"Adjusted all scores for agent {a_id}: {all_scores}")
    print(f"Selected location for agent {a_id}: {a_origin_2}")

# 출력하여 확인
print("Agent locations (agn_locs):")
print(agn_locs)


All indices for agent 0: [    0     1     2 ... 47037 47038 47039]
All values for agent 0: [0.9 0.9 0.9 ... 0.9 0.9 0.9]
Adjusted all scores for agent 0: [0.9 0.9 0.9 ... 0.9 0.9 0.9]
Selected location for agent 0: (32, 8, 5)
All indices for agent 1: [    0     1     2 ... 47037 47038 47039]
All values for agent 1: [0.9 0.9 0.9 ... 0.9 0.9 0.9]
Adjusted all scores for agent 1: [0.9 0.9 0.9 ... 0.9 0.9 0.9]
Selected location for agent 1: (40, 9, 5)
All indices for agent 2: [    0     1     2 ... 47037 47038 47039]
All values for agent 2: [0.9 0.9 0.9 ... 0.9 0.9 0.9]
Adjusted all scores for agent 2: [0.9 0.9 0.9 ... 0.9 0.9 0.9]
Selected location for agent 2: (33, 8, 5)
All indices for agent 3: [    0     1     2 ... 47037 47038 47039]
All values for agent 3: [0.8 0.8 0.8 ... 0.8 0.8 0.8]
Adjusted all scores for agent 3: [0.8 0.8 0.8 ... 0.8 0.8 0.8]
Selected location for agent 3: (39, 10, 5)
All indices for agent 4: [    0     1     2 ... 47037 47038 47039]
All values for agent 4: [0.5

In [7]:
# max_agents_per_floor = 6
# occ_lattice = avail_lattice * 0 - 1
# init_occ_lattice = np.copy(occ_lattice)

# agn_locs = [None] * len(program_prefs)  # 각 에이전트의 위치를 저장할 리스트
# floor_counts = np.zeros(avail_lattice.shape[2], dtype=int)  # 층별 에이전트 수를 추적하는 배열 초기화

# for a_id, a_prefs in program_prefs.iterrows():
#     pref_lattice = avail_lattice * 0.0
#     for f, w in a_prefs.items():
#         pref_lattice += fields[f] * w

#     flat_pref_lattice = pref_lattice.flatten()
    
#     # 모든 가용한 위치의 인덱스를 가져옴
#     all_indices = np.where(flat_pref_lattice > 0)[0]
    
#     # 각 가용한 위치에 대해 프로그램 간 상관관계를 반영하여 선호도 조정
#     all_scores = flat_pref_lattice[all_indices].copy()
#     adjustment_factors = np.ones_like(all_scores)
#     num_adjustments = np.zeros_like(all_scores)  # 각 위치에 대한 상관관계 적용 횟수를 추적

#     for other_id in range(len(program_prefs)):
#         if other_id != a_id and agn_locs[other_id] is not None:
#             corr = program_mtx.iloc[a_id, other_id]  # DataFrame에서 상관관계 값을 가져옴
#             other_loc = agn_locs[other_id]
#             for idx, loc in enumerate(np.unravel_index(all_indices, pref_lattice.shape)):
#                 if tuple(loc) == other_loc:
#                     adjustment_factors[idx] += corr
#                     num_adjustments[idx] += 1

#     # 가중평균 계산: 기본 선호도와 상관관계의 조정값을 가중평균으로 반영
#     for i in range(len(all_scores)):
#         if num_adjustments[i] > 0:
#             all_scores[i] = (all_scores[i] + adjustment_factors[i]) / (num_adjustments[i] + 1)

#     selected = False
#     a_origin_2 = None
#     for select_id in all_indices[np.argsort(-all_scores)]:  # 모든 위치에서 최적의 위치를 선택
#         a_origin_1 = np.unravel_index(select_id, avail_lattice.shape)
#         # 가용한 위치와 층의 에이전트 수를 고려하여 위치 선정
#         if avail_lattice[a_origin_1] != 0 and floor_counts[a_origin_1[2]] < max_agents_per_floor:
#             a_origin_2 = a_origin_1
#             selected = True
#             break

#     if not selected:
#         # 만약 모든 위치 중에서 적절한 위치를 찾지 못했을 때, 다음으로 높은 값을 찾음
#         while not selected:
#             select_id = np.argmax(flat_pref_lattice)
#             a_origin_1 = np.unravel_index(select_id, avail_lattice.shape)
#             if avail_lattice[a_origin_1] != 0 and floor_counts[a_origin_1[2]] < max_agents_per_floor:
#                 a_origin_2 = a_origin_1
#                 selected = True
#             else:
#                 flat_pref_lattice[a_origin_1] = -1  # 현재 위치를 -1로 설정하여 다음 최고 값을 선택하도록

#     # 에이전트 위치 리스트에 추가
#     agn_locs[a_id] = a_origin_2
#     # 가용 배열에서 해당 위치를 0으로 설정 (이제 사용 불가능)
#     avail_lattice[a_origin_2] = 0
#     # 해당 층의 에이전트 수를 증가
#     floor_counts[a_origin_2[2]] += 1
#     occ_lattice[a_origin_2] = a_id

#     print(f"All indices for agent {a_id}: {all_indices}")
#     print(f"All values for agent {a_id}: {flat_pref_lattice[all_indices]}")
#     print(f"Adjusted all scores for agent {a_id}: {all_scores}")
#     print(f"Selected location for agent {a_id}: {a_origin_2}")

# # 출력하여 확인
# print("Agent locations (agn_locs):")
# print(agn_locs)


All indices for agent 0: [    0     1     2 ... 47037 47038 47039]
All values for agent 0: [0.9 0.9 0.9 ... 0.9 0.9 0.9]
Adjusted all scores for agent 0: [0.9 0.9 0.9 ... 0.9 0.9 0.9]
Selected location for agent 0: (31, 8, 5)
All indices for agent 1: [    0     1     2 ... 47037 47038 47039]
All values for agent 1: [0.9 0.9 0.9 ... 0.9 0.9 0.9]
Adjusted all scores for agent 1: [0.9 0.9 0.9 ... 0.9 0.9 0.9]
Selected location for agent 1: (46, 9, 5)
All indices for agent 2: [    0     1     2 ... 47037 47038 47039]
All values for agent 2: [0.9 0.9 0.9 ... 0.9 0.9 0.9]
Adjusted all scores for agent 2: [0.9 0.9 0.9 ... 0.9 0.9 0.9]
Selected location for agent 2: (38, 10, 5)
All indices for agent 3: [    0     1     2 ... 47037 47038 47039]
All values for agent 3: [0.8 0.8 0.8 ... 0.8 0.8 0.8]
Adjusted all scores for agent 3: [0.8 0.8 0.8 ... 0.8 0.8 0.8]
Selected location for agent 3: (40, 10, 5)
All indices for agent 4: [    0     1     2 ... 47037 47038 47039]
All values for agent 4: [0.

In [8]:
import pyvista as pv
import numpy as np
from matplotlib.colors import ListedColormap

# Initialize PyVista plotter
p = pv.Plotter(notebook=True)

# Set the grid dimensions
grid = pv.ImageData()
grid.dimensions = np.array(occ_lattice.shape) + 1
grid.origin = avail_lattice.minbound - avail_lattice.unit * 0.5
grid.spacing = avail_lattice.unit

# Adding the bounding box wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")
p.add_axes()
p.show_bounds(grid="back", location="back", color="#777777")

# Add the data values to the cell data
grid.cell_data["Agents"] = occ_lattice.flatten(order="F").astype(int)  # Flatten the array!

# Filtering the voxels
agn_num = len(program_prefs)
threshed = grid.threshold([-0.1, agn_num - 0.9])

colors = [
    "#FFB6C1", "#FFC0CB", "#FF69B4", "#FF1493",  # 분홍색 계열
    "#87CEEB", "#00BFFF", "#1E90FF",  # 파란색 계열
    "#98FB98", "#00FF7F", "#32CD32", "#3CB371", "#2E8B57", "#228B22",  # 녹색 계열
    "#FFD700", "#FFA500", "#FF8C00", "#FF7F50", "#FF6347", "#FF4500", "#FF0000", "#DC143C", "#B22222",  # 노란색 계열
    "#BA55D3", "#9932CC", "#9400D3", "#8A2BE2", "#9370DB", "#7B68EE", "#6A5ACD",  # 자주색 계열
    "#FFE4B5", "#FFDAB9", "#FFDEAD",  # 베이지 계열
    "#F0E68C", "#EEE8AA", "#F5DEB3", "#DEB887", "#D2B48C", "#BC8F8F",  # 갈색 계열
    "#B0C4DE", "#ADD8E6", "#B0E0E6", "#AFEEEE", "#00CED1",  # 연한 파란색 계열
    "#696969",  # 회색 계열
    "#708090",  # 슬레이트 그레이 계열
    "#4682B4", "#5F9EA0", "#6495ED", "#7B68EE", "#87CEFA", "#87CEFA"  # 추가 파란색 계열
]

custom_cmap = ListedColormap(colors)

# Scalar bar 설정
sargs = dict(
    shadow=True,
    n_labels=agn_num,
    italic=False,
    fmt=" %.0f",
    font_family="arial",
    height=0.6,
    vertical=True,
    title_font_size=10,  # 제목 폰트 크기 설정
    label_font_size=6   # 라벨 폰트 크기 설정
)

annotations = {i: name for i, name in enumerate(space_list.values())}

# Add the voxels with the custom colormap and scalar bar arguments
mesh = p.add_mesh(threshed, name='sphere', show_edges=True, opacity=0.5, show_scalar_bar=True, annotations=annotations, scalar_bar_args=sargs, cmap=custom_cmap)

# Add other components
street_pc.fast_notebook_vis(p)
p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa', opacity=0.3)
p.add_mesh(entrance_pc, color='red', opacity=1.0)

# Selected program cell variable
selected_mesh_actor = None

# Slider callback function
def slider_callback(value):
    global selected_mesh_actor
    selected_program = int(value)
    mask = threshed.cell_data["Agents"] == selected_program
    opacity = np.full(threshed.n_cells, 0.5)  # Default opacity setting for non-selected cells
    opacity[mask] = 1.0  # Full opacity for selected program

    # Update opacity
    threshed.cell_data["opacity"] = opacity

    # Update text at bottom left
    p.add_text(f"Selected Program: {annotations[selected_program]}", position='lower_left', font_size=12, color='black', name="selected_text", render=True)

    # Remove previously selected mesh if exists
    if selected_mesh_actor is not None:
        p.remove_actor(selected_mesh_actor)

    # Highlight selected cells with red outline
    selected_cells = threshed.extract_cells(mask)
    selected_mesh_actor = p.add_mesh(selected_cells, color=None, show_edges=True, edge_color='red', line_width=8)

    # Update the main mesh to reflect opacity changes
    #mesh.update_scalars(threshed.cell_data["Agents"], render=True)
    mesh.GetProperty().SetOpacityArray(threshed.cell_data["opacity"])

    p.render()

# Slider widget addition
p.add_slider_widget(slider_callback, [0, len(program_prefs) - 1], value=0, title='Program Index', style='classic', pointa=(.1, .9), pointb=(.9, .9))

# Show the plot
p.show(jupyter_backend="trame", return_viewer=True)



c:\Users\junglim\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyvista\core\utilities\misc.py:207: UserWarning: Encountered issue in callback (most recent call last):
  File "C:\Users\junglim\AppData\Local\Temp\ipykernel_27792\3118067109.py", line 92, in slider_callback
    mesh.GetProperty().SetOpacityArray(threshed.cell_data["opacity"])
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'Property' object has no attribute 'SetOpacityArray'
  warnings.warn(formatted_exception)


Widget(value='<iframe src="http://localhost:60787/index.html?ui=P_0x2bc88efd010_0&reconnect=auto" class="pyvis…

c:\Users\junglim\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyvista\core\utilities\misc.py:207: UserWarning: Encountered issue in callback (most recent call last):
  File "C:\Users\junglim\AppData\Local\Temp\ipykernel_27792\3118067109.py", line 92, in slider_callback
    mesh.GetProperty().SetOpacityArray(threshed.cell_data["opacity"])
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'Property' object has no attribute 'SetOpacityArray'
  warnings.warn(formatted_exception)


In [6]:
lattice_cens = avail_lattice.centroids_threshold(-1)

def distance(a_id, fns):
    fns_cens = lattice_cens[np.ravel_multi_index(np.array(fns).T, avail_lattice.shape)] # 이웃하는 격자의 중심점을 가져옴 (벡터화된 연산으로 개선)
    agn_avg_loc = np.average(agn_locs[a_id], axis=0)                                    # 에이전트의 평균 위치 계산 (루프 밖에서 한 번만 계산)
    dist_m = np.sqrt(((fns_cens - agn_avg_loc)**2).sum(axis=1))                         # 평균 위치와의 차이를 기반으로 거리 계산 (벡터화된 연산으로 개선)

    return dist_m  # 계산된 거리 반환

In [7]:

# 주어진 스텐실을 기반으로 이웃을 캐싱하는 함수
def cache_neighbors(lattice, stencil):
    neighbors_cache = {}
    for x in range(lattice.shape[0]):  # x축을 따라 루프
        for y in range(lattice.shape[1]):  # y축을 따라 루프
            for z in range(lattice.shape[2]):  # z축을 따라 루프
                loc = (x, y, z)  # 현재 위치를 (x, y, z)로 지정
                # 주어진 스텐실을 사용하여 현재 위치에서 이웃을 찾고 캐시에 저장
                neighbors_cache[loc] = lattice.find_neighbours_masked(stencil, loc=loc, id_type="3D")
    return neighbors_cache

# 이웃 캐시 생성
neighbors_cache = cache_neighbors(avail_lattice, stencil)

In [8]:

# # 주어진 스텐실을 기반으로 이웃을 캐싱하는 함수
# def cache_neighbors(lattice, stencil):
#     neighbors_cache = {}
#     for x in range(lattice.shape[0]):  # x축을 따라 루프
#         for y in range(lattice.shape[1]):  # y축을 따라 루프
#             for z in range(lattice.shape[2]):  # z축을 따라 루프
#                 loc = (x, y, z)  # 현재 위치를 (x, y, z)로 지정
#                 # 주어진 스텐실을 사용하여 현재 위치에서 이웃을 찾고 캐시에 저장
#                 neighbors_cache[loc] = lattice.find_neighbours_masked(stencil, loc=loc, id_type="3D")
#     return neighbors_cache

# # 스텐실 정의
# stencil = tg.create_stencil("von_neumann", 1, 1)
# stencil_sq = tg.create_stencil("von_neumann", 1, 1)
# stencil_sq.set_index([0, 0, 0], 0)  # 중심 인덱스 비활성화
# stencil_sq.set_index([0, 0, 1], 0)  # z축 방향으로 비활성화
# stencil_sq.set_index([0, 0, -1], 0)  # z축 방향으로 비활성화

# # 이웃 캐시 생성
# neighbors_cache = cache_neighbors(avail_lattice, stencil)
# neighbors_sq_cache = cache_neighbors(avail_lattice, stencil_sq)

In [9]:
def evaluate_ratio(a_locs, n, target_ratio):
    # 현재 위치와 새로운 위치의 스팬을 계산
    new_locs = np.array(a_locs + [n])
    x_span = new_locs[:, 0].max() - new_locs[:, 0].min() + 1
    y_span = new_locs[:, 1].max() - new_locs[:, 1].min() + 1
    z_span = new_locs[:, 2].max() - new_locs[:, 2].min() + 1

    # 현재 스팬 비율을 계산하고 목표 비율과 비교
    span_diff_x_y = abs(x_span - y_span)
    span_diff_y_z = abs(y_span - z_span)
    span_diff_x_z = abs(x_span - z_span)

    # 스팬 차이의 합을 계산 (차이가 작을수록 목표 비율에 가깝다고 판단)
    span_diff_total = span_diff_x_y + span_diff_y_z + span_diff_x_z
    
    # 가중치를 계산 (차이가 클수록 가중치가 작아짐)
    weight = np.exp(-span_diff_total)
    
    return weight

target_ratio = (3, 2, 0.1)

In [10]:
def initialize_simulation():
    global occ_lattice, frames, t, n_frames, max_spaces
    occ_lattice = tg.to_lattice(np.copy(init_occ_lattice), occ_lattice)
    frames = [init_occ_lattice]
    t = 0
    n_frames = 50  # 시뮬레이션 프레임 수 설정
    # 각 에이전트의 최대 공간 크기를 미리 계산
    max_spaces = np.rint(agent_areas).astype(int)

# 시뮬레이션 초기화
initialize_simulation()


'외부 평가 (a_eval)': 에이전트가 현재 위치에서 이동할 수 있는 모든 가능한 새로운 위치(빈 이웃)에 대해 평가합니다. 즉, 에이전트가 새로 이동할 수 있는 모든 위치의 적합성을 계산하여, 각 위치가 얼마나 좋은지 평가합니다.

'내부 평가 (i_eval)': 에이전트가 이미 점유하고 있는 위치들에 대해 평가합니다. 즉, 에이전트가 현재 점유하고 있는 위치가 얼마나 좋은지 평가합니다. 이 평가를 통해 기존 위치 중에서 덜 적합한 위치를 제거할 수 있습니다.

In [11]:
# while t < n_frames:
#     n_fns = 0  # 빈 이웃이 없는 에이전트의 수를 세는 변수 초기화

#     # 각 에이전트의 선호도에 대해 반복
#     for a_id, a_prefs in program_prefs.iterrows():
#         if agn_locs[a_id] is None:
#             agn_locs[a_id] = []

#         # 현재 에이전트의 위치를 가져옴
#         a_locs = agn_locs[a_id]
#         a_locs = [np.unravel_index(loc, avail_lattice.shape) if isinstance(loc, int) else loc for loc in a_locs]

#         free_neighs = []

#         # 에이전트의 각 위치에 대한 루프
#         for a_loc in a_locs:
#             if isinstance(a_loc, tuple):  # a_loc이 튜플인지 확인
#                 neighs = neighbors_cache[a_loc]
#                 for n in neighs:
#                     if avail_lattice[tuple(n)]:
#                         free_neighs.append(tuple(n))

#         if not free_neighs:
#             n_fns += 1
#         else:
#             fns = np.array(free_neighs)
#             a_eval = np.zeros(fns.shape[0])
#             num_factors = len(program_prefs.columns) + len(program_mtx.columns) + 3  # 프로그램 선호도, 거리 평가, 비율 점수, 수평 및 수직 편향

#             # 프로그램 선호도 적용
#             for f in program_prefs.columns:
#                 a_eval += fields[f][fns[:, 0], fns[:, 1], fns[:, 2]] * a_prefs[f]

#             # 각 프로그램에 대한 거리와 가중치 평가
#             for s in program_mtx.columns:
#                 s = int(s)
#                 vals = distance(a_id, fns)  # 거리 값을 계산
#                 a_weighted_vals = vals * program_mtx.loc[a_id, s]
#                 a_eval += a_weighted_vals

#             # 비율 점수 계산
#             ratio_scores = np.zeros(len(free_neighs))
#             for i, n in enumerate(free_neighs):
#                 ratio_scores[i] = evaluate_ratio(a_locs, n, target_ratio)

#             # 수평 및 수직 점수 계산
#             horizontal_scores = np.zeros(len(free_neighs))
#             vertical_scores = np.zeros(len(free_neighs))
#             horizontal_bias = 2
#             vertical_penalty = 0.1
#             for i, n in enumerate(free_neighs):
#                 if abs(n[1] - a_locs[0][1]) > 0:
#                     horizontal_scores[i] = horizontal_bias
#                 else:
#                     horizontal_scores[i] = 1.0
#                 if abs(n[2] - a_locs[0][2]) > 0:
#                     vertical_scores[i] = vertical_penalty
#                 else:
#                     vertical_scores[i] = 1.0

#             # 가중평균 계산 및 적용
#             for i in range(len(free_neighs)):
#                 a_eval[i] = (a_eval[i] + ratio_scores[i] + horizontal_scores[i] + vertical_scores[i]) / num_factors

#             # 최적 위치 선택
#             selected_int = np.argmax(a_eval)

#             # 새로운 위치 추가
#             selected_neigh_3d_id = tuple(free_neighs[selected_int])
#             a_locs.append(selected_neigh_3d_id)
#             avail_lattice[selected_neigh_3d_id] = False
#             occ_lattice[selected_neigh_3d_id] = a_id

#             # 최대 공간 크기 초과 시 내부 평가로 선택하여 제거
#             current_length = len(a_locs)
#             max_space = max_spaces[a_id]  # 미리 계산된 최대 공간 크기 사용

#             if current_length > max_space:
#                 i_eval = np.zeros(current_length)

#                 for f in program_prefs.columns:
#                     vals = fields[f][np.array(a_locs)[:, 0], np.array(a_locs)[:, 1], np.array(a_locs)[:, 2]]
#                     a_weighted_vals = vals * a_prefs[f]
#                     i_eval += a_weighted_vals

#                 for s in program_mtx.columns:
#                     s = int(s)
#                     vals = distance(a_id, np.array(a_locs))
#                     a_weighted_vals = vals * program_mtx.loc[a_id, s]
#                     i_eval += a_weighted_vals

#                 selected_int_inner = np.argmin(i_eval)
#                 selected_inner_3d_id = tuple(a_locs[selected_int_inner])
#                 a_locs.pop(selected_int_inner)
#                 avail_lattice[selected_inner_3d_id] = True
#                 occ_lattice[selected_inner_3d_id] = -1

#         agn_locs[a_id] = a_locs
#         print(f"Agent {a_id} locations after update: {agn_locs[a_id]}")  # 디버깅 출력

#     new_occ_lattice = np.copy(occ_lattice)
#     frames.append(new_occ_lattice)
#     print(f"Frame {t}: {n_fns} agents without free neighbors")
#     t += 1

# print("Simulation completed.")


In [12]:
# # 시뮬레이션 루프
# while t < n_frames:  # 프레임 수 만큼 반복
#     n_fns = 0  # 빈 이웃이 없는 에이전트의 수를 세는 변수 초기화

#     for a_id, a_prefs in program_prefs.iterrows():  # 각 에이전트의 선호도에 대해 반복
#         if isinstance(agn_locs[a_id], (tuple, int)):
#             agn_locs[a_id] = [agn_locs[a_id]]

#         a_locs = agn_locs[a_id]
#         a_locs = [np.unravel_index(loc, avail_lattice.shape) if isinstance(loc, int) else loc for loc in a_locs]

#         free_neighs = []
#         free_neighs_sq = []

#         for a_loc in a_locs:
#             if avail_lattice.shape[0] > 0 and avail_lattice.shape[1] > 0 and avail_lattice.shape[2] > 0:
#                 neighs = neighbors_cache[tuple(a_loc)]
#                 neighs_sq = neighbors_sq_cache[tuple(a_loc)]
#             else:
#                 print("avail_lattice shape is invalid:", avail_lattice.shape)
#                 neighs = []
#                 neighs_sq = []

#             for n in neighs:
#                 if avail_lattice[tuple(n)]:
#                     free_neighs.append(tuple(n))

#             for n in neighs_sq:
#                 if avail_lattice[tuple(n)]:
#                     free_neighs_sq.append(tuple(n))

#         if not free_neighs:
#             n_fns += 1

#         if free_neighs:
#             fns = np.array(free_neighs)
#             a_eval = np.sum([fields[f][fns[:, 0], fns[:, 1], fns[:, 2]] * a_prefs[f] for f in program_prefs.columns], axis=0)

#             for s in program_mtx.columns:
#                 s = int(s)
#                 vals = distance(s, fns)
#                 a_weighted_vals = vals * program_mtx.loc[a_id, s]
#                 a_eval += a_weighted_vals

#             free_neighs_set = set(free_neighs)
#             free_neighs_sq_set = set(free_neighs_sq)
#             free_neighs_count = [sum(1 for n in free_neighs_sq if n in free_neighs_set) for _ in free_neighs]

#             square_weight = 1  # 수평 이동을 선호하도록 가중치를 더욱 높게 설정
#             a_weighted_square = np.array(free_neighs_count) * square_weight
#             a_eval *= a_weighted_square

#             horizontal_bias = 1  # 수평 방향으로의 가중치 설정
#             vertical_penalty = 0.2  # 수직 방향 이동에 대한 강한 패널티 설정
#             for i, n in enumerate(free_neighs):
#                 if abs(n[1] - a_locs[0][1]) > 0:
#                     a_eval[i] *= horizontal_bias
#                 if abs(n[2] - a_locs[0][2]) > 0:
#                     a_eval[i] *= vertical_penalty

#             for i, n in enumerate(free_neighs):
#                 ratio_score = evaluate_ratio(a_locs, n, target_ratio)
#                 a_eval[i] /= (1 + ratio_score)

#             current_length = len(a_locs)
#             i_eval = np.zeros(current_length)
#             max_space = np.rint(agent_areas[a_id]).astype(int)

#             if current_length >= max_space:
#                 i_eval = np.ones(current_length)

#                 for f in program_prefs.columns:
#                     vals = fields[f][np.array(a_locs)[:, 0], np.array(a_locs)[:, 1], np.array(a_locs)[:, 2]]
#                     a_weighted_vals = vals * a_prefs[f]
#                     i_eval *= a_weighted_vals

#                 for s in program_mtx.columns:
#                     s = int(s)
#                     vals = distance(s, np.array(a_locs))
#                     a_weighted_vals = vals * program_mtx.loc[a_id, s]
#                     i_eval *= a_weighted_vals

#                 i_neighs_count = np.zeros(current_length)

#                 for id, a_loc in enumerate(a_locs):
#                     neighs = neighbors_cache[tuple(a_loc)]
#                     for n in neighs:
#                         if avail_lattice[tuple(n)] and (occ_lattice[tuple(n)] == a_id):
#                             i_neighs_count[id] += 1

#                 i_weighted_square = i_neighs_count * square_weight
#                 i_eval *= i_weighted_square

#             selected_int_inner = np.argmin(i_eval)  # 내부 선택 인덱스 계산
#             selected_int = np.argmax(a_eval)  # 외부 선택 인덱스 계산

#             if current_length >= max_space and i_eval[selected_int_inner] < a_eval[selected_int]:  # 공간이 꽉 차고 내부 평가가 외부 평가보다 낮은 경우
#                 selected_inner_3d_id = tuple(a_locs[selected_int_inner])
#                 a_locs.pop(selected_int_inner)  # 내부 위치 제거
#                 avail_lattice[selected_inner_3d_id] = True
#                 occ_lattice[selected_inner_3d_id] = -1

#             if current_length < max_space:  # 공간이 꽉 차지 않은 경우
#                 selected_neigh_3d_id = tuple(free_neighs[selected_int])
#                 a_locs.append(selected_neigh_3d_id)  # 빈 이웃에 추가
#                 avail_lattice[selected_neigh_3d_id] = False
#                 occ_lattice[selected_neigh_3d_id] = a_id

#         agn_locs[a_id] = a_locs  # 업데이트된 위치 저장



#     new_occ_lattice = tg.to_lattice(np.copy(occ_lattice), occ_lattice)  # 새로운 격자 생성
#     frames.append(new_occ_lattice)  # 프레임에 추가
#     print(t, "/", n_fns, end="  ")  # 현재 프레임과 빈 이웃이 없는 에이전트 수 출력
#     t += 1  # 시간 증가

In [13]:
a_prefs

sun_acc        0.2
ent_acc        0.1
str_acc        0.1
ung_pre        0.8
dist_facade    0.1
top_pre        0.1
Name: 50, dtype: float64

In [14]:
a_pref_weight = 0.5
distance_weight = 0.5


for a_id, a_prefs in program_prefs.iterrows():
    a_locs = agn_locs[a_id]
    if isinstance(a_locs, tuple):
        a_locs = [a_locs]  # 단일 위치를 리스트로 변환
    
    free_neighs = []  # 빈 이웃 위치를 저장할 리스트 초기화

    for a_loc in a_locs:                    
        if avail_lattice.shape[0] > 0 and avail_lattice.shape[1] > 0 and avail_lattice.shape[2] > 0:
            try:
                neighs = neighbors_cache[tuple(a_loc)]  # 현재 위치의 이웃을 가져옴
                free_neighs.extend(neighs)
            except TypeError as e:                
                pass

    if free_neighs:
        fns = np.array(free_neighs)  # 빈 이웃을 배열로 변환

        # a_pref 기반 평가 계산
        a_eval_pref = np.sum([fields[f][fns[:, 0], fns[:, 1], fns[:, 2]] * a_prefs[f] for f in program_prefs.columns], axis=0)
        a_eval_pref_rounded = np.round(a_eval_pref, 2)  # 소수점 둘째 자리까지 반올림

        # 거리 기반 평가 계산
        a_eval_distance = np.zeros_like(a_eval_pref)  # 초기화
        for s in program_mtx.columns:
            s = int(s)
            vals = distance(s, fns)  # 거리 값을 계산
            # 거리 값을 리맵 (0과 1 사이로)
            scaler = MinMaxScaler()
            vals_scaled = scaler.fit_transform(vals.reshape(-1, 1)).flatten()
            a_weighted_vals = vals_scaled * program_mtx.loc[a_id, s]  # 가중치 적용
            a_eval_distance += a_weighted_vals  # 평가 값에 추가

        a_eval_distance_rounded = np.round(a_eval_distance, 2)  # 소수점 둘째 자리까지 반올림

        # 가중치 적용한 최종 평가 계산
        a_eval_final = (a_pref_weight * a_eval_pref_rounded) + (distance_weight * a_eval_distance_rounded)
        
        print(f"Initial a_eval for agent {a_id} (a_pref):\n", a_eval_pref_rounded)
        print(f"After distance evaluation for agent {a_id}:\n", a_eval_distance_rounded)
        print(f"Final a_eval for agent {a_id}:\n", np.round(a_eval_final, 2))


Initial a_eval for agent 0 (a_pref):
 [1.95 1.94 1.81 1.94 1.93 1.94]
After distance evaluation for agent 0:
 [11.33  5.69  0.   16.4   9.71  4.  ]
Final a_eval for agent 0:
 [6.64 3.82 0.9  9.17 5.82 2.97]
Initial a_eval for agent 1 (a_pref):
 [1.81 0.9  0.9  1.8  1.82 1.82]
After distance evaluation for agent 1:
 [ 8.71  6.22  0.   13.3   6.51  3.94]
Final a_eval for agent 1:
 [5.26 3.56 0.45 7.55 4.16 2.88]
Initial a_eval for agent 2 (a_pref):
 [1.61 0.9  0.9  1.58 1.6  1.6 ]
After distance evaluation for agent 2:
 [ 7.48  5.44  0.   11.6   5.7   3.59]
Final a_eval for agent 2:
 [4.54 3.17 0.45 6.59 3.65 2.6 ]
Initial a_eval for agent 3 (a_pref):
 [1.51 1.51 0.8  1.5  1.51 1.51]
After distance evaluation for agent 3:
 [14.4  10.04  0.   22.   11.01  6.52]
Final a_eval for agent 3:
 [ 7.96  5.78  0.4  11.75  6.26  4.01]
Initial a_eval for agent 4 (a_pref):
 [1.48 1.49 1.5  1.5  1.49 1.47]
After distance evaluation for agent 4:
 [15.99  9.87  0.   24.9  14.23  8.05]
Final a_eval for a

In [15]:
# 시뮬레이션 루프 - 지정된 프레임 수만큼 반복
while t < n_frames:  
    n_fns = 0  # 빈 이웃이 없는 에이전트의 수를 세는 변수 초기화

    # 각 에이전트의 선호도에 대해 반복
    for a_id, a_prefs in program_prefs.iterrows():  
        # agn_locs[a_id]가 리스트가 아닌 경우 리스트로 변환
        if isinstance(agn_locs[a_id], (tuple, int)):
            agn_locs[a_id] = [agn_locs[a_id]]

        # 현재 에이전트의 위치를 가져옴
        a_locs = agn_locs[a_id]
        # a_locs가 int형이면 이를 격자 인덱스로 변환      

        free_neighs = []  # 빈 이웃 위치를 저장할 리스트 초기화

        # 에이전트의 각 위치에 대한 루프
        for a_loc in a_locs:
            # 격자 크기가 유효한 경우
            if avail_lattice.shape[0] > 0 and avail_lattice.shape[1] > 0 and avail_lattice.shape[2] > 0:  
                neighs = neighbors_cache[tuple(a_loc)]  # 현재 위치의 이웃을 가져옴
            else:
                print("avail_lattice shape is invalid:", avail_lattice.shape)
                neighs = []  # 유효하지 않으면 빈 리스트로 설정
                
            # 빈 이웃 목록에 추가
            for n in neighs:
                if avail_lattice[tuple(n)]:  # 이웃이 비어 있으면
                    free_neighs.append(tuple(n))  # 튜플로 변환하여 추가

        # 빈 이웃이 없는 경우
        if not free_neighs:  
            n_fns += 1  # 빈 이웃이 없는 에이전트 수 증가

        # 빈 이웃이 있는 경우
        if free_neighs:  
            fns = np.array(free_neighs)  # 빈 이웃을 배열로 변환

            # 외부 평가: 각 빈 이웃에 대한 평가 값 계산
            a_eval = np.sum([fields[f][fns[:, 0], fns[:, 1], fns[:, 2]] * a_prefs[f] for f in program_prefs.columns], axis=0)

            # 각 프로그램에 대한 거리와 가중치 평가
            for s in program_mtx.columns:
                s = int(s)
                vals = distance(s, fns)  # 거리 값을 계산
                a_weighted_vals = vals * program_mtx.loc[a_id, s]  # 가중치 적용
                a_eval += a_weighted_vals  # 평가 값에 추가

            free_neighs_set = set(free_neighs)  # 빈 이웃을 집합으로 변환

                        # 비율 평가 함수 적용
            for i, n in enumerate(free_neighs):
                ratio_score = evaluate_ratio(a_locs, n, target_ratio)
                a_eval[i] *= ratio_score  # 비율 점수가 낮을수록 선호되도록 설정

            # 수평 이동을 선호하도록 가중치 적용
            horizontal_bias = 3 # 수평 방향으로의 가중치 설정
            vertical_penalty = 0.1  # 수직 방향 이동에 대한 패널티 설정
            for i, n in enumerate(free_neighs):
                if abs(n[1] - a_locs[0][1]) > 0:  # y축 방향으로 이동
                    a_eval[i] *= horizontal_bias
                if abs(n[2] - a_locs[0][2]) > 0:  # z축 방향으로 이동
                    a_eval[i] *= vertical_penalty      


            current_length = len(a_locs)  # 현재 에이전트가 점유한 공간의 길이
            i_eval = np.zeros(current_length)  # 내부 선택을 위한 평가 값 초기화
            max_space = np.rint(agent_areas[a_id]).astype(int)  # 최대 공간 크기 계산

            # 현재 길이가 최대 공간 이상인 경우
            if current_length >= max_space:  
                i_eval = np.ones(current_length)  # 내부 평가 값 초기화

                # 내부 평가: 현재 위치의 평가 값 계산
                for f in program_prefs.columns:
                    vals = fields[f][np.array(a_locs)[:, 0], np.array(a_locs)[:, 1], np.array(a_locs)[:, 2]]
                    a_weighted_vals = vals * a_prefs[f]
                    i_eval *= a_weighted_vals

                for s in program_mtx.columns:
                    s = int(s)
                    vals = distance(s, np.array(a_locs))
                    a_weighted_vals = vals * program_mtx.loc[a_id, s]
                    i_eval *= a_weighted_vals

                i_neighs_count = np.zeros(current_length)

                # 이웃 평가: 현재 위치의 이웃 수 계산
                for id, a_loc in enumerate(a_locs):
                    neighs = neighbors_cache[tuple(a_loc)]
                    for n in neighs:
                        if avail_lattice[tuple(n)] and (occ_lattice[tuple(n)] == a_id):
                            i_neighs_count[id] += 1

            # 내부 선택 인덱스 계산 (i_eval이 최소인 인덱스 선택)
            selected_int_inner = np.argmin(i_eval)
            # 외부 선택 인덱스 계산 (a_eval이 최대인 인덱스 선택)
            selected_int = np.argmax(a_eval)

            # 내부 평가가 외부 평가보다 낮은 경우
            if current_length >= max_space and i_eval[selected_int_inner] < a_eval[selected_int]:  
                selected_inner_3d_id = tuple(a_locs[selected_int_inner])
                a_locs.pop(selected_int_inner)  # 내부 위치 제거
                avail_lattice[selected_inner_3d_id] = True
                occ_lattice[selected_inner_3d_id] = -1

            # 공간이 꽉 차지 않은 경우
            if current_length < max_space:  
                selected_neigh_3d_id = tuple(free_neighs[selected_int])
                a_locs.append(selected_neigh_3d_id)  # 빈 이웃에 추가
                avail_lattice[selected_neigh_3d_id] = False
                occ_lattice[selected_neigh_3d_id] = a_id

        agn_locs[a_id] = a_locs  # 업데이트된 위치 저장

    new_occ_lattice = tg.to_lattice(np.copy(occ_lattice), occ_lattice)  # 새로운 격자 생성
    frames.append(new_occ_lattice)  # 프레임에 추가
    print(t, "/", n_fns, end="  ")  # 현재 프레임과 빈 이웃이 없는 에이전트 수 출력
    t += 1  # 시간 증가


0 / 0  1 / 0  2 / 0  3 / 0  4 / 0  5 / 0  6 / 0  7 / 0  8 / 0  9 / 0  10 / 0  11 / 0  12 / 0  13 / 0  14 / 0  15 / 0  16 / 0  17 / 0  18 / 0  19 / 0  20 / 0  21 / 0  22 / 0  23 / 0  24 / 0  25 / 0  26 / 0  27 / 0  28 / 0  29 / 1  30 / 1  31 / 1  32 / 1  33 / 1  34 / 1  35 / 1  36 / 1  37 / 1  38 / 1  39 / 1  40 / 1  41 / 1  42 / 1  43 / 1  44 / 1  45 / 1  46 / 1  47 / 1  48 / 1  49 / 1  

In [16]:
from matplotlib.colors import ListedColormap

# PyVista 플로터 초기화
p = pv.Plotter(notebook=True)

# 그리드 설정
#base_lattice = frames[0]
grid = pv.ImageData()
grid.dimensions = np.array(occ_lattice.shape) + 1
# grid.origin과 grid.spacing 설정에 avail_lattice의 속성 사용
grid.origin = avail_lattice.minbound - avail_lattice.unit * 0.5
grid.spacing = avail_lattice.unit

# 데이터 할당
grid.cell_data["Agents"] = occ_lattice.flatten(order="F").astype(int)

# 필터링
agn_num = len(program_prefs)
threshed = grid.threshold([-0.1, agn_num - 0.9])

# 초기 컬러맵 설정
initial_colors = ["#808080" if i == 0 else "#FF0000" for i in range(len(program_prefs))]
custom_cmap = ListedColormap(initial_colors)

# 스칼라 바 설정
annotations = {i: name for i, name in enumerate(space_list.values())}
sargs = {
    "shadow": True,
    "n_labels": len(program_prefs),
    "italic": False,
    "fmt": " %.0f",
    "font_family": "arial",
    "height": 1,
    "vertical": True,
    "title_font_size": 10,
    "label_font_size": 6
}

# 메시 추가
opacity_map = np.full(threshed.n_cells, 0.1)  # 초기 투명도 설정
threshed.cell_data['opacity'] = opacity_map
mesh = p.add_mesh(threshed, scalars='opacity', cmap=custom_cmap, show_scalar_bar=True, scalar_bar_args=sargs, show_edges=True, opacity=1)

# 텍스트 추가
text_actor = p.add_text("Selected Program: None", position='upper_right', font_size=12, color='black')

# 슬라이더 콜백 함수
def slider_callback(value):
    selected_program = int(value)
    mask = threshed.cell_data['Agents'] == selected_program
    opacity = np.full(threshed.n_cells, 0.1)  # 기본 투명도 설정
    opacity[mask] = 1.0  # 선택된 프로그램에 높은 투명도 부여

    # 투명도 업데이트
    threshed.cell_data['opacity'] = opacity
    
    # 텍스트 업데이트
    text_actor.SetText(0, f"Selected Program: {annotations[selected_program]}")
    p.render()

# 슬라이더 위젯 추가
p.add_slider_widget(slider_callback, [0, len(program_prefs) - 1], value=0, title='Program Index')
p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=True, annotations=annotations, scalar_bar_args=sargs, cmap=custom_cmap)

p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa', opacity=0.3)
p.add_mesh(entrance_pc, color='red', opacity=1.0)
# 시각화 표시
p.show(jupyter_backend="trame", return_viewer=True)

Widget(value='<iframe src="http://localhost:50266/index.html?ui=P_0x251a3a8ff50_1&reconnect=auto" class="pyvis…

In [17]:
from matplotlib.colors import ListedColormap
import pyvista as pv

# Visualization setup
p = pv.Plotter(notebook=True)

# Set the grid dimensions
grid = pv.ImageData()
grid.dimensions = np.array(occ_lattice.shape) + 1
# grid.origin과 grid.spacing 설정에 avail_lattice의 속성 사용
grid.origin = avail_lattice.minbound - avail_lattice.unit * 0.5
grid.spacing = avail_lattice.unit

# Adding the bounding box wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")
p.add_axes()
p.show_bounds(grid="back", location="back", color="#777777")

# Add the data values to the cell data
grid.cell_data["Agents"] = occ_lattice.flatten(order="F").astype(int)  # Flatten the array!

# Filtering the voxels
agn_num = len(program_prefs)
threshed = grid.threshold([-0.1, agn_num - 0.9])

colors = [
    "#FFB6C1", "#FFC0CB", "#FF69B4", "#FF1493",  # 분홍색 계열
    "#87CEEB", "#00BFFF", "#1E90FF",  # 파란색 계열
    "#98FB98", "#00FF7F", "#32CD32", "#3CB371", "#2E8B57", "#228B22",  # 녹색 계열
    "#FFD700", "#FFA500", "#FF8C00", "#FF7F50", "#FF6347", "#FF4500", "#FF0000", "#DC143C", "#B22222",  # 노란색 계열
    "#BA55D3", "#9932CC", "#9400D3", "#8A2BE2", "#9370DB", "#7B68EE", "#6A5ACD",  # 자주색 계열
    "#FFE4B5", "#FFDAB9", "#FFDEAD",  # 베이지 계열
    "#F0E68C", "#EEE8AA", "#F5DEB3", "#DEB887", "#D2B48C", "#BC8F8F",  # 갈색 계열
    "#B0C4DE", "#ADD8E6", "#B0E0E6", "#AFEEEE", "#00CED1",  # 연한 파란색 계열
    "#696969",  # 회색 계열
    "#708090",  # 슬레이트 그레이 계열
    "#4682B4", "#5F9EA0", "#6495ED", "#7B68EE", "#87CEFA", "#87CEFA"  # 추가 파란색 계열
]

custom_cmap = ListedColormap(colors)
# Scalar bar 설정
sargs = dict(
    shadow=True,
    n_labels=agn_num,
    italic=False,
    fmt=" %.0f",
    font_family="arial",
    height=0.6,
    vertical=True,

    title_font_size=10,  # 제목 폰트 크기 설정
    label_font_size=6   # 라벨 폰트 크기 설정
)
annotations = {i: name for i, name in enumerate(space_list.values())}
# Adding the voxels with the custom colormap and scalar bar arguments
p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=True, annotations=annotations, scalar_bar_args=sargs, cmap=custom_cmap)
street_pc.fast_notebook_vis(p)
p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa', opacity=0.3)
p.add_mesh(entrance_pc, color='red', opacity=1.0)
#envelope_lattice.fast_vis(p)
# Show the plot
p.show(jupyter_backend="trame", return_viewer=True)  # 시각화 표시


Widget(value='<iframe src="http://localhost:50266/index.html?ui=P_0x251a3a34f50_2&reconnect=auto" class="pyvis…

Exception raised
KeyError('e777a6959edbab1f841c260af7dba6d0_642f')
Traceback (most recent call last):
  File "c:\Users\junglim\AppData\Local\Programs\Python\Python311\Lib\site-packages\wslink\protocol.py", line 308, in onCompleteMessage
    results = func(*args, **kwargs)
              ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\junglim\AppData\Local\Programs\Python\Python311\Lib\site-packages\trame_vtk\modules\vtk\protocols\local_rendering.py", line 33, in get_array
    self.context.get_cached_data_array(data_hash, binary)
  File "c:\Users\junglim\AppData\Local\Programs\Python\Python311\Lib\site-packages\trame_vtk\modules\vtk\serializers\synchronization_context.py", line 35, in get_cached_data_array
    cache_obj = self.data_array_cache[p_md5]
                ~~~~~~~~~~~~~~~~~~~~~^^^^^^^
KeyError: 'e777a6959edbab1f841c260af7dba6d0_642f'

Exception raised
KeyError('e5ccf9fba391f04fc6cfcda94fa94f3c_970L')
Traceback (most recent call last):
  File "c:\Users\junglim\AppData\Local\Programs\Pyth